In [1]:
from __future__ import print_function
import os
import re
import numpy as np
import numpy
genecode_genes = set(os.listdir("experiment_blood"))
from scipy.stats import pearsonr
from sklearn.cross_decomposition import PLSRegression
import random
import bisect
import math
import json
import sys
import sklearn.linear_model
import sklearn.gaussian_process
import sklearn.ensemble

In [2]:
def in_ipynb():
    try:
        get_ipython()
        return True
    except NameError:
        return False

In [3]:
if in_ipynb():
    from matplotlib import pyplot as plt

In [4]:
print("Please use like this: `predictions.py <tissue> <genes> <simulation_runs> <top_k_genes> <model>`\n"\
      "tissue can be any of: blood, muscle.\n"\
      "genes can be any of: TNNI1, batra, nakamori, all\n"\
      "simulation runs can be 10\n"\
      "top_k_genes can be 500\n"\
      "model can be any of: LASSO, LinearRegression, RandomForestRegressor, PLSR"
     )
if not in_ipynb():
    tissue_arg = sys.argv[1]
    print("tissue: {}".format(tissue_arg))
    gene_arg = sys.argv[2]
    print("gene: {}".format(gene_arg))
    simulation_arg = int(sys.argv[3])
    print("simulation_arg: {}".format(simulation_arg))
    top_genes_arg = int(sys.argv[4])
    print("top_genes_arg: {}".format(top_genes_arg))
    model_arg = sys.argv[5]
    print("model_arg: {}".format(model_arg))

Please use like this: `predictions.py <tissue> <genes> <simulation_runs> <top_k_genes> <model>`
tissue can be any of: blood, muscle.
genes can be any of: TNNI1, batra, nakamori, all
simulation runs can be 10
top_k_genes can be 500
PLSR_dimensions can be 2


In [5]:
def load_genes(filename):
    genes = set()
    repeated = set()
    with open(filename) as f:
        for line in f:
            line = line.rstrip()
            if line in genes:
                repeated.add(line)
            genes.add(line)
    failed = genes.difference(genecode_genes)
    if repeated:
        pass
        #print("These genes appear more than once: {}".format([g for g in repeated]))
    if failed:
        print("Couldn't identify the following genes: {}".format([g for g in failed]))
    return genes.intersection(genecode_genes)

In [6]:
nakamori_genes = load_genes("nakamori_genes.txt")

In [7]:
# Genes from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4224598/
batra_genes = load_genes("batra_genes.txt")

In [8]:
class Metadata:
    def __init__(self, ids, cels, metadata):
        self.IDs = ids
        self.CELs = cels
        self.modal_allele = [int(metadata[i]["modal_allele"]) for i in ids]
        self.progenitor_allele = [int(metadata[i]["progenitor_allele"]) for i in ids]
        self.MIRS = [int(metadata[i]["MIRS"]) for i in ids]
    def __str__(self):
        return "Metadata={{IDs: {}...,\n CELs: {}...,\n modal_allele: {}...,\n progenitor_allele: {}...,\n MIRS: {}...}}".format(self.IDs[:5], self.CELs[:5], self.modal_allele[:5], self.progenitor_allele[:5], self.MIRS[:5])
    def __repr__(self):
        return self.__str__()

def load_metadata():
    metadata = {}
    metadata_order = []
    with open("metadata.txt") as f:
        for i, line in enumerate(f):
            line = line.strip().split()
            if i == 0:
                names = line[1:]
            else:
                values = line[1:]
                patient_id = line[0]
                metadata_order.append(patient_id)
                metadata[patient_id] = {k: v for k, v in zip(names, values)}
    blood_IDs = [i for i in metadata_order]
    muscle_IDs = [i for i in metadata_order if metadata[i]["muscle_cel"] != "refused_biopsy"]
    blood_CELs = [metadata[i]["blood_cel"] for i in blood_IDs]
    muscle_CELs = [metadata[i]["muscle_cel"] for i in muscle_IDs]
        
    blood_record = Metadata(blood_IDs, blood_CELs, metadata)
    muscle_record = Metadata(muscle_IDs, muscle_CELs, metadata)
    return blood_record, muscle_record

In [9]:
blood_meta, muscle_meta = load_metadata()

In [10]:
def produce_data(meta, gene_names, experiment):
    IDs = meta.IDs
    probe_data = []
    probe_IDs = []
    for gene in gene_names:        
        with open(os.path.join(experiment, gene)) as f:
            for i, line in enumerate(f):
                line = line.rstrip().split()
                if i == 0:
                    prefix = "patient_"
                    our_IDs = [elem[len(prefix):] for elem in line if re.match(prefix, elem)]
                    try:
                        assert IDs == our_IDs
                    except AssertionError:
                        print(gene)
                    headers = {header: i for i, header in enumerate(line)}
                    patient_data = {header[len(prefix):]: i for i, header in enumerate(line) if re.match(prefix, header)}
                    def write_signature(line):
                        signature = []
                        for elem in ["gene_name", "probeset_id", "seq5to3plus", "chrom", "strand", "genocode_left", "genecode_right"]:
                            signature.append(line[headers[elem]])
                        return "_".join(signature)
                else:
                    probe_ID = write_signature(line)
                    rv = []
                    for patient_id in IDs:
                        rv.append(float(line[patient_data[patient_id]]))
                    probe_data.append(rv)
                    probe_IDs.append(probe_ID)
    probe_data = numpy.array(probe_data)
    return probe_data, probe_IDs

In [11]:
def choose_top_genes(indicies, data):
    """
    Take `indicies`, which is a list of ordered pairs
    `r2, column_index`, and a matrix of probeset data,
    `data`, and filter all rows to contain only genes on
    the `indicies` list.
    """
    just_indices = [i[1] for i in indicies]
    return data[just_indices, :]

def order_indicies(allele, data):
    """
    Return a sorted list of correlated genes, from the most to the least
    correlated. Each entry in the list is a tuple `r2, gene_index`, where
    `r2` denotes the correlation of gene expression and `allele`, and
    the index of the gene in the `data` matrix respectively.
    
    `allele` is the list of repeat lengths, with each repeat length
    corresponding to one column in `data`
    """
    r2s = []
    for i in range(data.shape[0]):
        gene_r, _ = pearsonr(allele, data[i])
        # We have to store both the gene and the gene index, to recover the indices after sorting.
        r2s.append((gene_r ** 2, i))
    r2s.sort()
    # This reverses the list, assuring decreasing order.
    r2s = r2s[::-1]
    return r2s

def choose_training_indicies(data, no_training_samples):
    """Return a partition of column indicies of the `data`
    matrix into two sets: `training_indicies, testing_indicies`.
    The training set will contain `no_training_samples` biological
    samples.
    """
    indices = list(range(data.shape[1]))
    random.shuffle(indices)
    training_indices = indices[:no_training_samples]
    test_indicies = indices[no_training_samples:]
    return training_indices, test_indicies

def split_data(data, training_indices, test_indices):
    """Return a partition of data into `training_set, testing_set`
    using an existing partition of column incidices into
    `training_indicies` and `test_indicies`
    """
    return data[:, training_indices], data[:, test_indices]

def cv(lista):
    return np.matrix(lista).transpose()
def rv(lista):
    return np.matrix(lista)

In [12]:
def filter_extremes(value, lowermost, uppermost):
    """If a value is lower or higher than lowermost or uppermost, return lowermost or uppermost. Otherwise, return value.
    """
    if value < lowermost:
        return lowermost
    if value > uppermost:
        return uppermost
    return value

In [13]:
def simulation_step_PLSR(probeset_data, metadata, training_size, no_probes):
    n_components = 2
    """Execute a full step of the simulation, and return a tuple consisting of two lists:
    predicted repeat lengths, and the ground truth repeat lengths
    """
    training_i, testing_i = choose_training_indicies(probeset_data, training_size)
    training_fold, testing_fold = split_data(probeset_data, training_i, testing_i)
    repeat_length_training = [metadata.modal_allele[i] for i in training_i]
    repeat_length_testing = [metadata.modal_allele[i] for i in testing_i]
    
    top_genes = order_indicies(repeat_length_training, training_fold)[:no_probes]
    training_set = choose_top_genes(top_genes, training_fold)
    testing_set = choose_top_genes(top_genes, testing_fold)

    plsr = PLSRegression(n_components=n_components)
    
    plsr.fit(training_set.transpose(), cv(repeat_length_training))
    
    results = plsr.predict(testing_set.transpose())
    results = [filter_extremes(i[0], min(repeat_length_training), max(repeat_length_training)) for i in results]

    return repeat_length_testing, results

In [14]:
def simulation_step_LASSO(probeset_data, metadata, training_size, no_probes):
    n_components = 10**(-6)
    training_i, testing_i = choose_training_indicies(probeset_data, training_size)
    training_fold, testing_fold = split_data(probeset_data, training_i, testing_i)
    repeat_length_training = [metadata.modal_allele[i] for i in training_i]
    repeat_length_testing = [metadata.modal_allele[i] for i in testing_i]
    
    top_genes = order_indicies(repeat_length_training, training_fold)[:no_probes]
    training_set = choose_top_genes(top_genes, training_fold)
    testing_set = choose_top_genes(top_genes, testing_fold)

    lasso = sklearn.linear_model.Lasso(alpha=n_components)
    
    lasso.fit(training_set.transpose(), cv(repeat_length_training))
    
    results = list(lasso.predict(testing_set.transpose()))
    
    results = [filter_extremes(i, min(repeat_length_training), max(repeat_length_training)) for i in results]

    return repeat_length_testing, results

In [15]:
def simulation_step_LinearRegression(probeset_data, metadata, training_size, no_probes):
    training_i, testing_i = choose_training_indicies(probeset_data, training_size)
    training_fold, testing_fold = split_data(probeset_data, training_i, testing_i)
    repeat_length_training = [metadata.modal_allele[i] for i in training_i]
    repeat_length_testing = [metadata.modal_allele[i] for i in testing_i]
    
    top_genes = order_indicies(repeat_length_training, training_fold)[:no_probes]
    training_set = choose_top_genes(top_genes, training_fold)
    testing_set = choose_top_genes(top_genes, testing_fold)

    model = sklearn.linear_model.LinearRegression()
    
    model.fit(training_set.transpose(), cv(repeat_length_training))
    
    results = model.predict(testing_set.transpose())
    results = [i for i in results.transpose()[0]]
    
    results = [filter_extremes(i, min(repeat_length_training), max(repeat_length_training)) for i in results]
    return repeat_length_testing, results

In [16]:
def simulation_step_GaussianProcess(probeset_data, metadata, training_size, no_probes):
    training_i, testing_i = choose_training_indicies(probeset_data, training_size)
    training_fold, testing_fold = split_data(probeset_data, training_i, testing_i)
    repeat_length_training = [metadata.modal_allele[i] for i in training_i]
    repeat_length_testing = [metadata.modal_allele[i] for i in testing_i]
    
    top_genes = order_indicies(repeat_length_training, training_fold)[:no_probes]
    training_set = choose_top_genes(top_genes, training_fold)
    testing_set = choose_top_genes(top_genes, testing_fold)

    model = sklearn.gaussian_process.GaussianProcessRegressor()
    
    model.fit(training_set.transpose(), cv(repeat_length_training))
    
    results = model.predict(testing_set.transpose())
    results = [i for i in results.transpose()[0]]
    
    return repeat_length_testing, results

In [17]:
def simulation_step_RandomForestRegressor(probeset_data, metadata, training_size, no_probes):
    training_i, testing_i = choose_training_indicies(probeset_data, training_size)
    training_fold, testing_fold = split_data(probeset_data, training_i, testing_i)
    repeat_length_training = [metadata.modal_allele[i] for i in training_i]
    repeat_length_testing = [metadata.modal_allele[i] for i in testing_i]
    
    top_genes = order_indicies(repeat_length_training, training_fold)[:no_probes]
    training_set = choose_top_genes(top_genes, training_fold)
    testing_set = choose_top_genes(top_genes, testing_fold)

    model = sklearn.ensemble.RandomForestRegressor()
    
    model.fit(training_set.transpose(), repeat_length_training)
    
    results = list(model.predict(testing_set.transpose()))
    
    return repeat_length_testing, results

In [18]:
assert filter_extremes(0, 10, 20) == 10
assert filter_extremes(15, 10, 20) == 15
assert filter_extremes(25, 10, 20) == 20

In [19]:
metadata = {"muscle": muscle_meta, "blood": blood_meta}

In [20]:
metadata

{'muscle': Metadata={IDs: ['111747589', '117440822', '124563003', '129523253', '159834720']...,
  CELs: ['111747589_M.CEL', '117440822_M.CEL', '124563003_M.CEL', '129523253_M.CEL', '159834720_MR.cel']...,
  modal_allele: [872, 297, 593, 408, 1035]...,
  progenitor_allele: [600, 227, 301, 211, 695]...,
  MIRS: [4, 2, 4, 4, 4]...},
 'blood': Metadata={IDs: ['111747589', '117440822', '124563003', '129523253', '141772399']...,
  CELs: ['111747589_B.CEL', '117440822_B.CEL', '124563003_B.CEL', '129523253_B.CEL', '141772399_B.CEL']...,
  modal_allele: [872, 297, 593, 408, 717]...,
  progenitor_allele: [600, 227, 301, 211, 320]...,
  MIRS: [4, 2, 4, 4, 2]...}}

In [21]:
def load_data(name, genes, muscle_or_blood):
    meta = metadata[muscle_or_blood]
    path = "experiment_" + muscle_or_blood
    d, a = produce_data(meta, genes, path)
    data[muscle_or_blood][name] = d
    annot[muscle_or_blood][name] = a

In [22]:
gene_sets = {"nakamori": nakamori_genes, "batra": batra_genes, "TNNI1": ["TNNI1"], "all": genecode_genes}

In [23]:
data = {"muscle": {}, "blood": {}}
annot = {"muscle": {}, "blood": {}}

In [24]:
for name, genes in gene_sets.items():
    for tissue in ["blood", "muscle"]:
        load_data(name, genes, tissue)

In [25]:
training_size = int(len(metadata["muscle"].modal_allele)*0.7)

In [26]:
testing_size = len(metadata["muscle"].modal_allele) - training_size

In [27]:
def simulation(simulation_step, data, metadata, runs, no_probes):
    sample_no, patient_no = data.shape
    real_alleles = []
    simulated_alleles = []
    for i in range(runs):
        real_allele, simulated_allele = simulation_step(data, metadata, int(patient_no*0.7), no_probes)
        real_alleles += real_allele
        simulated_alleles += simulated_allele
    return real_alleles, simulated_alleles

In [28]:
def obtain_simulation(simulation, *args, **kwargs):
    real_alleles, simulated_alleles = simulation(*args, **kwargs)
    def save_at(path):
        with open(path, "w") as f:
            json.dump({"real_alleles": real_alleles,
                       "predicted_alleles": simulated_alleles,
                      }, f)
    return save_at

In [29]:
def analyse_simulation(path, r2_to_p):
    with open(path, "r") as f:
        result = json.load(f)
    real_alleles = result["real_alleles"]
    simulated_alleles = result["predicted_alleles"]
    r, _ = pearsonr(real_alleles, simulated_alleles)
    r_squared = r**2
    p_value = r2_to_p(r_squared)
    return real_alleles, simulated_alleles, r_squared, p_value, 

In [30]:
if in_ipynb():
    %matplotlib inline

In [31]:
def simulated_r2(metadata, times):
    r2s = []
    for i in range(times):
        repeat_length_random = metadata.modal_allele[:]
        random.shuffle(repeat_length_random)
        inter_r, _ = pearsonr(metadata.modal_allele, repeat_length_random)
        r2s.append(inter_r**2)
    r2s.sort()
    def check_value(r_squared):
        p_value = 1 - bisect.bisect_right(r2s, r_squared)/float(times)
        return p_value
    return check_value

In [32]:
base_path = "simulation_results"
try:
    os.mkdir(base_path)
except OSError:
    pass

In [33]:
def example_workflow(name, tissue, simulation_step, simulation_runs, select_top_k_genes):
    path = os.path.join(base_path, "{}_{}_{}_{}_{}".format(name, tissue, simulation_runs, select_top_k_genes, simulation_step.__name__))
    obtain_simulation(simulation, simulation_step, data[tissue][name], metadata[tissue], simulation_runs, select_top_k_genes)(path)
    print(path)

In [34]:
#simulation_step_LASSO(data["muscle"]["TNNI1"], metadata["muscle"], training_size, 500, 10**(-6))
#simulation_step_LinearRegression(data["muscle"]["TNNI1"], metadata["muscle"], training_size, 500, None)
#simulation_step_RandomForestRegressor(data["muscle"]["nakamori"], metadata["muscle"], training_size, 500, None)

In [35]:
models = {"LASSO": simulation_step_LASSO, "LinearRegression": simulation_step_LinearRegression, "RandomForestRegressor": simulation_step_RandomForestRegressor, "PLSR": simulation_step_PLSR}

In [36]:
if not in_ipynb():
    # For example
    # nohup python3 08_predictions.py muscle TNNI1 10 500 PLSR &
    example_workflow(name=gene_arg, tissue=tissue_arg, simulation_step=models[model_arg], simulation_runs=simulation_arg, select_top_k_genes=top_genes_arg)
else:
    example_workflow(name="TNNI1", tissue="muscle", simulation_step=simulation_step_PLSR, simulation_runs=10, select_top_k_genes=500)

simulation_results/TNNI1_muscle_10_500_simulation_step_PLSR
